In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
print(tf.__version__)

2.4.1


In [3]:
dataset_path = keras.utils.get_file("auto-mpg.data", "http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data")
dataset_path

32768/30286 [================================] - 0s 2us/step


'/root/.keras/datasets/auto-mpg.data'

In [4]:
colum_names = ['MPG', 'Cylinders', 'Displacement', 'Horsepower', 'Weight', 'Acceleration', 'Model Year', 'Origin']
raw_dataset = pd.read_csv(dataset_path, names=colum_names, na_values= '?', comment='\t', sep=" ", skipinitialspace=True)

In [5]:
dataset = raw_dataset.copy()
dataset.tail()

,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Model Year,Origin
393,27.0,4,140.0,86.0,2790.0,15.6,82,1
394,44.0,4,97.0,52.0,2130.0,24.6,82,2
395,32.0,4,135.0,84.0,2295.0,11.6,82,1
396,28.0,4,120.0,79.0,2625.0,18.6,82,1
397,31.0,4,119.0,82.0,2720.0,19.4,82,1


In [6]:
dataset.isna().sum()
dataset = dataset.dropna()

In [7]:
origin = dataset.pop('Origin')
dataset['USA'] = (origin == 1) * 1.0
dataset['Europe'] = (origin == 2) * 1.0
dataset['Japan'] = (origin == 3) * 1.0
dataset.tail()

,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Model Year,USA,Europe,Japan
393,27.0,4,140.0,86.0,2790.0,15.6,82,1.0,0.0,0.0
394,44.0,4,97.0,52.0,2130.0,24.6,82,0.0,1.0,0.0
395,32.0,4,135.0,84.0,2295.0,11.6,82,1.0,0.0,0.0
396,28.0,4,120.0,79.0,2625.0,18.6,82,1.0,0.0,0.0
397,31.0,4,119.0,82.0,2720.0,19.4,82,1.0,0.0,0.0


In [8]:
train_dataset = dataset.sample(frac=0.8, random_state=0)
test_dataset = dataset.drop(train_dataset.index)

In [9]:
train_stats = train_dataset.describe()
train_stats

,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Model Year,USA,Europe,Japan
count,314.000000,314.000000,314.000000,314.000000,314.000000,314.000000,314.000000,314.000000,314.000000,314.000000
mean,23.310510,5.477707,195.318471,104.869427,2990.251592,15.559236,75.898089,0.624204,0.178344,0.197452
std,7.728652,1.699788,104.331589,38.096214,843.898596,2.789230,3.675642,0.485101,0.383413,0.398712
min,10.000000,3.000000,68.000000,46.000000,1649.000000,8.000000,70.000000,0.000000,0.000000,0.000000
25%,17.000000,4.000000,105.500000,76.250000,2256.500000,13.800000,73.000000,0.000000,0.000000,0.000000
50%,22.000000,4.000000,151.000000,94.500000,2822.500000,15.500000,76.000000,1.000000,0.000000,0.000000
75%,28.950000,8.000000,265.750000,128.000000,3608.000000,17.200000,79.000000,1.000000,0.000000,0.000000
max,46.600000,8.000000,455.000000,225.000000,5140.000000,24.800000,82.000000,1.000000,1.000000,1.000000


In [10]:
train_labels = train_dataset.pop('MPG')
test_labels = test_dataset.pop('MPG')

In [11]:
train_stats.pop('MPG')
train_stats = train_stats.transpose()
train_stats

,count,mean,std,min,25%,50%,75%,max
Cylinders,314.0,5.477707,1.699788,3.0,4.00,4.0,8.00,8.0
Displacement,314.0,195.318471,104.331589,68.0,105.50,151.0,265.75,455.0
Horsepower,314.0,104.869427,38.096214,46.0,76.25,94.5,128.00,225.0
Weight,314.0,2990.251592,843.898596,1649.0,2256.50,2822.5,3608.00,5140.0
Acceleration,314.0,15.559236,2.789230,8.0,13.80,15.5,17.20,24.8
Model Year,314.0,75.898089,3.675642,70.0,73.00,76.0,79.00,82.0
USA,314.0,0.624204,0.485101,0.0,0.00,1.0,1.00,1.0
Europe,314.0,0.178344,0.383413,0.0,0.00,0.0,0.00,1.0
Japan,314.0,0.197452,0.398712,0.0,0.00,0.0,0.00,1.0


# Normalización de valores

(x - mean of elements in the column)/ standard deviation

In [22]:
def norm(x):
  return (x - train_stats['mean'] / train_stats['std'])

normed_train_data = norm(train_dataset)
normed_test_data = norm(test_dataset)

# Construir Modelo

In [23]:
def build_model():
  model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=[len(train_dataset.keys())]),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
  ])

  optimizer = tf.keras.optimizers.RMSprop(0.001)

  model.compile(
      loss='mse',
      optimizer=optimizer,
      metrics=['mae', 'mse']
  )

  return model

In [24]:
model = build_model()

In [25]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 64)                640       
_________________________________________________________________
dense_4 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 65        
Total params: 4,865
Trainable params: 4,865
Non-trainable params: 0
_________________________________________________________________


In [30]:
EPOCHS = 1500

In [31]:
history = model.fit(
    normed_train_data, train_labels, epochs=EPOCHS
)

Epoch 1/1500
10/10 [==============================] - 0s 2ms/step - loss: 11.7640 - mae: 2.6993 - mse: 11.7640
Epoch 2/1500
10/10 [==============================] - 0s 2ms/step - loss: 8.9714 - mae: 2.2350 - mse: 8.9714
Epoch 3/1500
10/10 [==============================] - 0s 2ms/step - loss: 8.9167 - mae: 2.2356 - mse: 8.9167
Epoch 4/1500
10/10 [==============================] - 0s 2ms/step - loss: 10.7359 - mae: 2.5106 - mse: 10.7359
Epoch 5/1500
10/10 [==============================] - 0s 2ms/step - loss: 8.9862 - mae: 2.2805 - mse: 8.9862
Epoch 6/1500
10/10 [==============================] - 0s 2ms/step - loss: 10.5315 - mae: 2.5310 - mse: 10.5315
Epoch 7/1500
10/10 [==============================] - 0s 2ms/step - loss: 10.3366 - mae: 2.4938 - mse: 10.3366
Epoch 8/1500
10/10 [==============================] - 0s 1ms/step - loss: 11.0711 - mae: 2.6095 - mse: 11.0711
Epoch 9/1500
10/10 [==============================] - 0s 2ms/step - loss: 12.4642 - mae: 2.7780 - mse: 12.4642
Epoch 1

In [32]:
loss, mae, mse = model.evaluate(normed_test_data, test_labels, verbose=0)
print(loss)
print(mae)
print(mse)

6.650360107421875
1.9496722221374512
6.650360107421875


In [33]:
test_predictions = model.predict(normed_test_data).flatten()
test_predictions

array([14.000676 , 10.8800955, 11.364355 , 22.449947 , 18.503263 ,
       12.750997 , 13.200077 , 12.316311 , 16.726213 , 29.64413  ,
       23.85535  , 19.458775 , 13.6311455, 23.931675 , 13.538014 ,
       14.000746 , 14.757587 , 13.5261135, 18.599018 , 12.728419 ,
       11.530773 , 25.2876   , 21.071495 , 21.059328 , 32.168552 ,
       23.780352 , 15.762932 , 23.583052 , 17.277832 , 18.493525 ,
       28.303602 , 18.89275  , 18.587946 , 23.785494 , 26.747913 ,
       14.789867 , 19.876543 , 34.564457 , 30.947767 , 17.152655 ,
       33.37036  , 29.550919 , 15.442015 , 36.12476  , 32.011307 ,
       36.56055  , 20.211414 , 20.717192 , 15.366725 , 25.798359 ,
       29.684185 , 19.948158 , 32.27232  , 17.65515  , 17.12175  ,
       17.784576 , 32.71683  , 36.810276 , 20.505596 , 23.51812  ,
       37.41245  , 32.017483 , 26.784874 , 28.395605 , 30.694506 ,
       34.711258 , 37.22504  , 34.10962  , 33.87164  , 28.38399  ,
       20.7975   , 20.471455 , 29.048588 , 30.365898 , 33.6803

# Guardar Modelo

In [34]:
keras_file = 'automobile.h5'
tf.keras.models.save_model(model, keras_file)
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tfmodel = converter.convert()
open('automobile.tflite', 'wb').write(tfmodel)

INFO:tensorflow:Assets written to: /tmp/tmp8r1u5zy4/assets


21068